In [122]:
# !pip install grad-cam

In [123]:
import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import json
from sklearn.metrics import roc_curve, auc

In [124]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cuda'

In [125]:
result_dict = {}
prob_dict = {}
df_global = []

In [126]:
def make_predictions(logs_path, technic):

    basedir = f'/home/eferreira/master/storage/ff++/ff++/test/manipulated_sequences/{technic}'

    videos = os.listdir(basedir)
    filenames = [x for x in videos]
    labels_df = [[x, 1] for x in filenames]

    original_seq_base_dir = "/home/eferreira/master/storage/ff++/ff++/test/original_sequences"
    original_videos = os.listdir(original_seq_base_dir)
    actors_filenames = [[x, 0] for x in original_videos]
    labels_df.extend(actors_filenames)
    labels_df = pd.DataFrame(labels_df, columns=['filename', 'label'])

    all_videos = os.listdir(basedir)
    all_videos = [os.path.join(basedir, x) for x in all_videos]

    original_faces_path = original_seq_base_dir
    original_faces = os.listdir(original_faces_path)
    original_faces = [os.path.join(original_faces_path, x) for x in original_faces]
    all_videos.extend(original_faces)

    save_dir = logs_path
    
    df = labels_df.copy()
    # print(df)
    # df.columns = ['label', 'filename']
    # df['id'] = df['filename'].apply(lambda x: x[:-4])
    df['filename'] = df['filename'].map(lambda x: x)
    df = df.sort_values(by='filename')
    json_file = os.path.join(logs_path, f"results_prob_{technic}.json")
    # print(json_file)
    # y_pred = df_filtered['prediction']
    with open(json_file, "r") as file:
        prob_dict = json.load(file)

    df['prediction_prob'] = df['filename'].map(prob_dict) 
    df_filtered = df[df['filename'].isin(prob_dict.keys())]
    
    

    # Calculando as métricas
    # print(df_filtered['filename'], result_dict)

    y_true = df_filtered['label']
    # print(df_filtered)
    y_pred_prob = df_filtered['prediction_prob']

    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    results = pd.read_csv(os.path.join(logs_path, f"metrics_{technic}.csv")).reset_index()
    # Modificar o valor de AUC
    results.loc[results['Metric'] == 'AUC', 'Value'] = roc_auc  # Substitua 0.85 pelo novo valor desejado

    # Salvar o CSV atualizado
    results.to_csv(os.path.join(logs_path, f'metrics_{technic}_new.csv'), index=False)


    # metrics_df = pd.DataFrame(results)
    print(results)
    print(results['Metric'])
    
    # metrics_df.to_csv(os.path.join(logs_path, f'metrics_{technic}_new.csv'), index=False)


    # print(results)

    #     # Plotando e salvando o gráfico da curva ROC
    # plt.figure()
    # plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    # plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    # plt.xlim([0.0, 1.0])
    # plt.ylim([0.0, 1.05])
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title('Receiver Operating Characteristic')
    # plt.legend(loc='lower right')
    # plt.savefig(os.path.join(save_dir, f'roc_curve_{technic}_new.png'))
    # plt.show()


In [127]:
log_path = ['/home/eferreira/master/cross-vit/CrossViT/old_logs/12_srm_input_transform',
            '/home/eferreira/master/cross-vit/CrossViT/old_logs/17_sobel_input_transform',
            '/home/eferreira/master/cross-vit/CrossViT/old_logs/18_baseline_0_5',
            '/home/eferreira/master/cross-vit/CrossViT/old_logs/24_srm_l_branch_simple',
            '/home/eferreira/master/cross-vit/CrossViT/old_logs/28_sobel_l_branch',
            # '/home/eferreira/master/cross-vit/CrossViT/old_logs/30_srm_l_branch',
            "/home/eferreira/master/cross-vit/CrossViT/old_logs/31_srm_s_branch",
            "/home/eferreira/master/cross-vit/CrossViT/old_logs/32_sobel_s_branch"
            ]
            
# log_path = ['/home/eferreira/master/cross-vit/CrossViT/old_logs/12']

In [128]:
# technic = 'DeepFakeDetection'
# basedir = f'/home/eferreira/master/storage/ff++/ff++/manipulated_sequences/{technic}/raw/videos'
# source_dir = os.path.join(basedir, "faces")

# videos = os.listdir(basedir)
# filenames = [x[:-4] for x in videos if 'mp4' in x]
# labels_df = [[x, 1] for x in filenames]
# original_seq_base_dir = "/home/eferreira/master/storage/ff++/ff++/original_sequences"
# original_actors = os.path.join(original_seq_base_dir, "actors/raw/videos")

# original_youtube = os.path.join(original_seq_base_dir, "youtube/raw/videos")
# actors_videos = os.listdir(original_actors)
# youtube_videos = os.listdir(original_youtube)
# actors_filenames = [[x[:-4], 0] for x in actors_videos if 'mp4' in x]
# youtube_filenames = [[x[:-4], 0] for x in youtube_videos if 'mp4' in x]
# labels_df.extend(actors_filenames)
# labels_df.extend(youtube_filenames)

In [129]:
for path in log_path:
    
    for technic in os.listdir('/home/eferreira/master/storage/ff++/ff++/manipulated_sequences'):
        print(technic)
        make_predictions(path, technic)

NeuralTextures
   index     Metric     Value
0      0   Accuracy  0.533571
1      1  Precision  0.535769
2      2     Recall  0.502857
3      3   F1 Score  0.518791
4      4        AUC  0.532471
5      5    Logloss  0.956067
0     Accuracy
1    Precision
2       Recall
3     F1 Score
4          AUC
5      Logloss
Name: Metric, dtype: object
Deepfakes
   index     Metric     Value
0      0   Accuracy  0.710714
1      1  Precision  0.660851
2      2     Recall  0.865714
3      3   F1 Score  0.749536
4      4        AUC  0.819457
5      5    Logloss  0.600958
0     Accuracy
1    Precision
2       Recall
3     F1 Score
4          AUC
5      Logloss
Name: Metric, dtype: object
DeepFakeDetection
   index     Metric     Value
0      0   Accuracy  0.732857
1      1  Precision  0.672669
2      2     Recall  0.907143
3      3   F1 Score  0.772506
4      4        AUC  0.844988
5      5    Logloss  0.572330
0     Accuracy
1    Precision
2       Recall
3     F1 Score
4          AUC
5      Logloss
N